In [1]:
#https://github.com/zalandoresearch/flair
!pip install --upgrade pip
!pip install flair

    100% |████████████████████████████████| 1.4MB 15.7MB/s ta 0:00:01
  Found existing installation: pip 19.0.3
    Uninstalling pip-19.0.3:
      Successfully uninstalled pip-19.0.3
     |████████████████████████████████| 81kB 3.8MB/s eta 0:00:011
     |████████████████████████████████| 133kB 10.3MB/s eta 0:00:01
     |████████████████████████████████| 614kB 30.1MB/s eta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/70/55/6b/c12cf58209b8346f653a04f37dd8f607ab0e85a26238a23420
  Stored in directory: /tmp/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /tmp/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Stored in directory: /tmp/.cache/pip/wheels/74/17/3f/c77bba99efd74ba1a19862c9dd97f4b6d735e2826721dc00ff
Successfully built flair segtok sqlitedict regex
  Found existing installation: regex 2019.4.14
    Uninstalling regex-2019.4.14:
      Successfully uninstalled regex-2019.4.14


Flair is:

    A powerful NLP library. Flair allows you to apply our state-of-the-art natural language processing (NLP) models to your text, such as named entity recognition (NER), part-of-speech tagging (PoS), sense disambiguation and classification.

    Multilingual. Thanks to the Flair community, we support a rapidly growing number of languages. We also now include 'one model, many languages' taggers, i.e. single models that predict PoS or NER tags for input text in various languages.

    A text embedding library. Flair has simple interfaces that allow you to use and combine different word and document embeddings, including our proposed Flair embeddings, BERT embeddings and ELMo embeddings.

    A Pytorch NLP framework. Our framework builds directly on Pytorch, making it easy to train your own models and experiment with new approaches using Flair embeddings and classes.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
from datetime import datetime   
    
PATH = "../input/cityofla/CityofLA/Job Bulletins/"

In [3]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('ner-ontonotes-fast') 

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


2019-05-17 11:21:07,537 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.2/NER-ontoner--h256-l1-b32-%2Bcrawl%2Bnews-forward-fast%2Bnews-backward-fast--v0.2/en-ner-ontonotes-fast-v0.3.pt not found in cache, downloading to /tmp/tmpn281t_b6


100%|██████████| 1331337776/1331337776 [00:57<00:00, 23070273.72B/s]

2019-05-17 11:22:05,798 copying /tmp/tmpn281t_b6 to cache at /tmp/.flair/models/en-ner-ontonotes-fast-v0.3.pt


2019-05-17 11:22:09,037 removing temp file /tmp/tmpn281t_b6
2019-05-17 11:22:09,046 loading file /tmp/.flair/models/en-ner-ontonotes-fast-v0.3.pt


In [4]:
def extract_ner(text, TERM='LAW'):
    
    sentence = Sentence(text)
    tagger.predict(sentence)
    r = sentence.to_dict(tag_type='ner')
    
    some_list = []
    for j in range(len(r['entities'])):
        if (r['entities'][j].get("type")) == TERM:
            some_list.append(r['entities'][j].get('text'))
    return some_list

In [5]:
text = []
for fl in tqdm(list(os.listdir("../input/cityofla/CityofLA/Job Bulletins"))):
    with open(PATH+fl, 'r',encoding='latin-1') as f: 
        text.append(f.read())
data = pd.DataFrame({'text':text})

100%|██████████| 683/683 [00:02<00:00, 284.54it/s]


In [6]:
data['title'] = data.text.apply(lambda x: x.split('\n')[0])
data['title'] = data.title.apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ',x))

In [7]:
data.text = data.text.apply(lambda x: x.replace('\t','').split('\n'))
data.text = data.text.apply(lambda x: ' '.join(x))

In [ ]:
data['law'] = data.text.apply(lambda x: extract_ner(x))
data['org'] = data.text.apply(lambda x: extract_ner(x, TERM='ORG'))
data['date'] = data.text.apply(lambda x: extract_ner(x, TERM='DATE'))
data['cardinal'] = data.text.apply(lambda x: extract_ner(x, TERM='CARDINAL'))
data['gpe'] = data.text.apply(lambda x: extract_ner(x, TERM='GPE'))
data['event'] = data.text.apply(lambda x: extract_ner(x, TERM='EVENT'))
data['percent'] = data.text.apply(lambda x: extract_ner(x, TERM='PERCENT'))
data['work_of_art'] = data.text.apply(lambda x: extract_ner(x, TERM='WORK_OF_ART'))

In [ ]:
data.head(10)

In [ ]:
data.to_csv('vacancy_ner.csv', index=False)